In [38]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import yaml, os

In [39]:
# ============================================================
# Spark
# ============================================================
spark = (
    SparkSession.builder
    .appName("silver_s1_make_tojidaejang_list")
    .master("spark://spark-master:7077")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.shuffle.partitions", "200")
    .getOrCreate()
)

26/02/19 12:49:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Config

In [40]:
# ============================================================
# Config
# ============================================================

CONFIG_PATH = "./config.yaml"

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

ROOT = cfg["data_lake"]["root"]
LAYERS = cfg["data_lake"]["layers"]

# Input Path
restaurant_src_path = os.path.join(
    ROOT,
    LAYERS["bronze"]["domains"]["restaurant_owner"]["paths"]["parquet"]
)

s0_address_path = os.path.join(
    ROOT,
    LAYERS["silver"]["stages"]["s0"]["domains"]["address"]["paths"]["parquet"]
)

s0_toji_building_path = os.path.join(
    ROOT,
    LAYERS["silver"]["stages"]["s0"]["domains"]["toji_building"]["paths"]["parquet"]
)

# Output Path
s1_crawling_list_path = os.path.join(
    ROOT,
    LAYERS["silver"]["stages"]["s1"]["domains"]["crawling_list"]["paths"]["parquet"]
)

s1_toji_list_path = os.path.join(
    ROOT,
    LAYERS["silver"]["stages"]["s1"]["domains"]["toji_list"]["paths"]["parquet"]
)

print("[PATH] restaurant_src_path  =", restaurant_src_path)
print("[PATH] s0_address_path      =", s0_address_path)
print("[PATH] s0_toji_building_path      =", s0_toji_building_path)
print("[PATH] s1_crawling_list_path   =", s1_crawling_list_path)
print("[PATH] s1_toji_list_path   =", s1_toji_list_path)

[PATH] restaurant_src_path  = /opt/spark/data/bronze/restaurant_owner/parquet
[PATH] s0_address_path      = /opt/spark/data/silver/s0/address
[PATH] s0_toji_building_path      = /opt/spark/data/silver/s0/toji_building
[PATH] s1_crawling_list_path   = /opt/spark/data/silver/s1/crawling_list
[PATH] s1_toji_list_path   = /opt/spark/data/silver/s1/toji_list


# 데이터 로드 (Bronze 식당 정보 + S0 주소, 토지_건축물)

In [42]:
rest_df = (
    spark.read.parquet(restaurant_src_path)
    .filter(F.col("region") == REGION)
    .select("업체명", "대표자", "소재지")
)

addr_df = (
    spark.read.parquet(s0_address_path)
    .filter(
        (F.col("region") == REGION)
    )
    .select("PNU코드", "도로명주소", "longitude", "latitude")
)

toji_building_df = (
    spark.read.parquet(s0_toji_building_path)
    .filter(
        (F.col("region") == REGION)
    )
)


print("rest_df")
rest_df.printSchema()
rest_df.show(2)
print("addr_df")
addr_df.printSchema()
addr_df.show(2)
print("toji_building_df")
toji_building_df.printSchema()
toji_building_df.show(2)

rest_df
root
 |-- 업체명: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 소재지: string (nullable = true)

+------------+------+--------------------------------+
|      업체명|대표자|                          소재지|
+------------+------+--------------------------------+
|        깜꼬| 강*영|   경기도 시흥시 신천3길 48(1...|
|니얼굴은혜씨| 장*실|경기도 양평군 강상면 강상로 3...|
+------------+------+--------------------------------+
only showing top 2 rows
addr_df
root
 |-- PNU코드: string (nullable = true)
 |-- 도로명주소: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)

+-------------------+---------------------------+------------------+------------------+
|            PNU코드|                 도로명주소|         longitude|          latitude|
+-------------------+---------------------------+------------------+------------------+
|4122010100108000000|경기도 평택시 경기대로 1366|127.06430001570745|37.066645122825705|
|4122010100107790003|경기도 평택시 경기대로 1401|127.06404931325756| 37.070

# 식당 데이터 정제

In [24]:
clean_rest_df = (
    rest_df
    # -----------------------------
    # 1. 괄호 제거
    # -----------------------------
    .withColumn(
        "소재지",
        F.regexp_replace(F.col("소재지"), r"\s*\([^)]*\)", "")
    )

    # -----------------------------
    # 2. 쉼표 뒤 제거
    # -----------------------------
    .withColumn(
        "소재지",
        F.regexp_replace(F.col("소재지"), r",.*$", "")
    )

    # -----------------------------
    # 3. 도로명 앞 행정동 제거
    # (고림동 고림로 123 → 고림로 123)
    # -----------------------------
    .withColumn(
        "소재지",
        F.regexp_replace(
            F.col("소재지"),
            r'\s+\S+(동|읍|면|리)\s+(?=\S+(로|길))',
            ' '
        )
    )
        
    # -----------------------------
    # 4. 공백 정리
    # -----------------------------
    .withColumn(
        "소재지",
        F.trim(F.regexp_replace(F.col("소재지"), r'\s+', ' '))
    )

    # -----------------------------
    # 대표자 정규화
    #   - "최*인 외 3인" -> "최*인"
    #   - "홍길동(대표)" -> "홍길동"
    #   - "홍길동, 김철수" -> "홍길동"
    #   - 공백 정리
    # -----------------------------
    .withColumn("대표자", F.trim(F.col("대표자")))
    # 괄호/부가설명 제거
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s*\([^)]*\)\s*", ""))
    # 쉼표/슬래시 등 다인 표기 -> 첫 토큰만 남기기
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s*[,/].*$", ""))
    # '외 N인/명' 제거 (붙어있는 케이스 포함)
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s*외\s*\d+\s*(인|명)\s*$", ""))
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s*외\s*\d+\s*$", ""))  # "외3" 같은 케이스
    # 끝에 남는 불필요 문자 정리
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s+", " "))
    .withColumn("대표자", F.trim(F.col("대표자")))

    # ------------------------------------
    # 시군구 필터링
    # ------------------------------------
    .withColumn("sigungu_tmp", F.regexp_extract(F.col("소재지"), r"\s([가-힣]+(?:시|군|구))\s", 1))
    .filter(F.col("sigungu_tmp") == SIGUNGU)
    .drop("sigungu_tmp")
)

clean_rest_df.show(2)

+------------+------+-----------------------------+
|      업체명|대표자|                       소재지|
+------------+------+-----------------------------+
|    쉼_,카페| 정*복|      경기도 평택시 평택로 51|
|자연한식뷔페| 정*화|경기도 평택시 고덕국제대로 77|
+------------+------+-----------------------------+
only showing top 2 rows


# 식당 + 주소 join

In [32]:
# ============================================================
# 식당 + 주소 join (matched + unmatched 같이 만들기)
# ============================================================

total = clean_rest_df.count()

joined_all_df = (
    clean_rest_df.alias("r")
    .join(
        addr_df.alias("a"),
        F.col("r.소재지") == F.col("a.도로명주소"),
        "left"
    )
    # 참고: r.소재지를 지울거면 디버깅이 어려워서
    # matched_df 만든 뒤에 drop하는 걸 추천
)

# ---- 디버깅용 unmatched ----
unmatched_df = (
    joined_all_df
    .filter(F.col("a.PNU코드").isNull())
    .select(
        F.col("r.업체명").alias("업체명"),
        F.col("r.대표자").alias("대표자"),
        F.col("r.소재지").alias("소재지"),
    )
)

# ---- 최종 matched ----
joined_rest_df = (
    joined_all_df
    .filter(F.col("a.PNU코드").isNotNull())
    .select(
        F.col("r.업체명").alias("업체명"),
        F.col("r.대표자").alias("대표자"),
        F.col("a.PNU코드").alias("PNU코드"),
        F.col("a.도로명주소").alias("도로명주소"),
        F.col("a.longitude").alias("longitude"),
        F.col("a.latitude").alias("latitude"),
    )
)

matched = joined_rest_df.count()
unmatched = total - matched

print("total     :", total)
print("matched   :", matched)
print("unmatched :", unmatched)
print("match rate:", matched / total if total else 0)

# ---- unmatched 샘플 출력 ----
print("\n===== UNMATCHED SAMPLE =====")
unmatched_df.show(5, truncate=False)

# ---- matched 샘플 ----
print("\n===== MATCHED SAMPLE =====")
joined_rest_df.show(5, truncate=False)


total     : 12364
matched   : 12237
unmatched : 127
match rate: 0.9897282432869622

===== UNMATCHED SAMPLE =====


+----------------------+------+------------------------------------+
|업체명                |대표자|소재지                              |
+----------------------+------+------------------------------------+
|송원푸드평택          |이*미 |경기도 평택시 현덕면 운정리 6-31 1층|
|송원푸드평택          |이*미 |경기도 평택시 현덕면 운정리 6-31 1층|
|효성함바식당(현장식당)|배*근 |경기도 평택시 모곡동 60-1 외6필지   |
|양반촌                |최*년 |경기도 평택시 청원로 1481           |
|마루                  |박*희 |경기도 평택시 신장동 294-11         |
+----------------------+------+------------------------------------+
only showing top 5 rows

===== MATCHED SAMPLE =====


[Stage 153:======================================>                  (2 + 1) / 3]

+-------------------------+-----------------+-------------------+---------------------------+------------------+------------------+
|업체명                   |대표자           |PNU코드            |도로명주소                 |longitude         |latitude          |
+-------------------------+-----------------+-------------------+---------------------------+------------------+------------------+
|평택시송탄출장소구내식당 |평*시송탄출장소장|4122010100108000000|경기도 평택시 경기대로 1366|127.06430001570745|37.066645122825705|
|모이라이카페 송탄출장소점|김*수            |4122010100108000000|경기도 평택시 경기대로 1366|127.06430001570745|37.066645122825705|
|최네집                   |양*웅            |4122010100107790003|경기도 평택시 경기대로 1401|127.06404931325756|37.07023131309042 |
|배스킨라빈스 송탄서정점  |이*복            |4122010100108170005|경기도 평택시 관광특구로 37|127.06294985880601|37.06694208205054 |
|맘스터치송탄점           |고*전            |4122010100108170005|경기도 평택시 관광특구로 37|127.06294985880601|37.06694208205054 |
+-------------------------+-----------------+-------------------+-----

In [43]:
spark.stop()